In [2]:
import tensorflow
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

In [83]:
trainFolder = "/dsvm/Notebooks/data/Train/"
y_train = np.loadtxt(trainFolder + "y_train.txt")
X_train = np.loadtxt(trainFolder + "X_train.txt")
id_train = np.loadtxt(trainFolder + "subject_id_train.txt")

In [86]:
testFolder = "/dsvm/Notebooks/data/Test/"
X_test = np.loadtxt(testFolder + "X_test.txt")
y_test = np.loadtxt(testFolder + "y_test.txt")
id_test = np.loadtxt(testFolder + "subject_id_test.txt")


In [5]:
np.shape(X_train), np.shape(y_train)

((7767, 561), (7767,))

In [35]:
set(y_train)

{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0}

In [6]:
chars = sorted(list(set(y_train)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
chars

total chars: 12


[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0]

In [47]:
maxlen = 1
step = 1
sentences = []
next_chars = []
for i in range(0, len(y_train), step):
    sentences.append(X_train[i: i + maxlen])
    next_chars.append(y_train[i])
print('nb sequences:', len(sentences))
print(sentences[:3], next_chars[:3])

nb sequences: 7767
[array([[ 0.04357967, -0.00597022, -0.03505434, -0.99538116, -0.98836586,
        -0.93738201, -0.99500705, -0.98881558, -0.9533252 , -0.79479637,
        -0.74489282, -0.64844725,  0.84179557,  0.70844018,  0.65171649,
        -0.97575203, -0.99995022, -0.99988837, -0.99801367, -0.99399929,
        -0.99197962, -0.97097005, -0.54709538, -0.70097364, -0.62269711,
         0.92188415, -0.71948288,  0.34216801, -0.16131765,  0.26604897,
        -0.27435145,  0.26720509, -0.02095843,  0.38261024, -0.50174806,
         0.51246321, -0.20633687,  0.376778  ,  0.43517232,  0.66019853,
         0.96005068, -0.13593871,  0.11555554, -0.98813447, -0.98269284,
        -0.91972268, -0.98836191, -0.98552296, -0.93183378,  0.89205451,
        -0.16135045,  0.12465977,  0.97743631, -0.12151281,  0.05611428,
        -0.37446422,  0.89182148, -0.97083555, -0.97551074, -0.98914081,
        -0.99044431, -0.94888696, -1.        , -1.        ,  0.03424856,
        -0.59314621,  0.5946358

In [9]:
np.shape(sentences)

(7766, 1, 561)

In [48]:
print('Vectorization...')
X = sentences
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    y[i, char_indices[next_chars[i]]] = 1
X = np.array(X)
X_train_vec = X
y_train_vec = y

Vectorization...


In [49]:
print("prepare vector for test data")
sentences = []
next_chars = []
for i in range(0, len(y_test), step):
    sentences.append(X_test[i: i + maxlen])
    next_chars.append(y_test[i])
print('nb vec_x:', len(sentences))
X = sentences
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    y[i, char_indices[next_chars[i]]] = 1
X = np.array(X)
X_test_vec = X
y_test_vec = y

prepare vector for test data
nb vec_x: 3162


In [12]:
np.shape(sentences), np.shape(next_chars), np.shape(X), np.shape(y)

((7766, 1, 561), (7766,), (7766, 1, 561), (7766, 12))

In [11]:
# build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(32, return_sequences=True, input_shape=(maxlen, 561)))
model.add(LSTM(32, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

Build model...


In [10]:
np.shape(X_train_vec), np.shape(y)

((7766, 1, 561), (7766, 12))

In [90]:
model.fit(X_train_vec, y_train_vec, batch_size = 128, nb_epoch= 20)
y_pred_test_vec = model.predict(X_test_vec)
y_pred_train_vec = model.predict(X_train_vec)
y_pred_test = getLabelFromVec(y_pred_test_vec)
y_pred_train = getLabelFromVec(y_pred_train_vec)
getAccuracy(y_test, y_pred_test)
getAccuracy(y_train, y_pred_train)

Epoch 1/20
7767/7767 [==============================] - 16s - loss: 5.6997e-04    
Epoch 2/20
7767/7767 [==============================] - 16s - loss: 0.0036    
Epoch 3/20
7767/7767 [==============================] - 17s - loss: 0.0057    
Epoch 4/20
7767/7767 [==============================] - 15s - loss: 1.1499e-04    
Epoch 5/20
7767/7767 [==============================] - 16s - loss: 0.0048    
Epoch 6/20
7767/7767 [==============================] - 15s - loss: 0.0051    
Epoch 7/20
7767/7767 [==============================] - 16s - loss: 0.0018    
Epoch 8/20
7767/7767 [==============================] - 16s - loss: 0.0013    
Epoch 9/20
7767/7767 [==============================] - 16s - loss: 0.0047    
Epoch 10/20
7767/7767 [==============================] - 16s - loss: 0.0052    
Epoch 11/20
7767/7767 [==============================] - 16s - loss: 0.0161    
Epoch 12/20
7767/7767 [==============================] - 16s - loss: 4.5339e-04    
Epoch 13/20
7767/7767 [==============

1.0

In [82]:
y_pred_test_vec = model.predict(X_test_vec)
y_pred_train_vec = model.predict(X_train_vec)
y_pred_test = getLabelFromVec(y_pred_test_vec)
y_pred_train = getLabelFromVec(y_pred_train_vec)
getAccuracy(y_test, y_pred_test)
getAccuracy(y_train, y_pred_train)

accuracy: 0.9459203036053131
accuracy: 1.0


1.0

In [33]:
def getAccuracy(y_test, y_pred_test):
    correct = (y_test == y_pred_test)
    accuracy = np.count_nonzero(correct)/len(correct)
    print("accuracy:", accuracy)
    return accuracy
def getLabelFromVec(y_pred_test_vec):
    indices = np.apply_along_axis(np.argmax, 1, y_pred_test_vec)   
    y_pred_test = np.array([indices_char[x] for x in indices])
    return y_pred_test